### find a city

In [2]:
import requests
import sqlite3
import pandas as pd

# NOAA API token
api_token = 'rDZYpxCxcNrIUzeserZMcYDaaHCNPcFZ'

# connect to the DB
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')

### extract data for desired cities

In [8]:
pd.read_sql_query('''
SELECT count(*)
from results
where city in(select SUBSTR(name, 1, INSTR(name, ',') - 1) from locations)
''' , conn)

,count(*)
0,32389


In [10]:
pd.read_sql_query('''
select count(*)
from Merged
''', conn)

,count(*)
0,33242


In [27]:
# join between locations and results (the city where the match played)
test = pd.read_sql_query('''
with part1 as (
SELECT 1 as locations,
SUBSTR(name, 1, INSTR(name, ',') - 1) as city_locations
FROM locations
),
part2 as (
SELECT distinct city as city_results, 1 as results
FROM results
),
part3 as (
select t1.*, t2.*
from part1 t1
full join part2 t2
on t1.city_locations = t2.city_results
)
''', conn)
test.to_csv('C:/Users/Mark Rozenberg/downloads/check1-v2.csv')

In [19]:
conn.execute(''' drop table if exists Merged''')
conn.commit()

In [26]:
conn.execute(''' drop table if exists Merged''')
conn.execute('''
create table if not exists merged as
with part1 as (
SELECT date, city as city_results
FROM results
),
part2 as (
SELECT id,
SUBSTR(name, 1, INSTR(name, ',') - 1) as city_locations,
mindate, maxdate
FROM locations
),
part3 as(
select t1.*, t2.*,
case when date between mindate and maxdate then 1 else 0 end as valid_date
from part1 t1
inner join part2 t2
on t1.city_results = t2.city_locations
)
select *
from part3
where valid_date = 1
''')
conn.commit()

In [13]:
test = pd.read_sql_query('''
SELECT *
FROM locations
where name like '%Barcelona%'
limit 10
''', conn)
test

,id,name,mindate,maxdate,datacoverage
0,CITY:SP000001,"Barcelona, SP",1913-08-01,2024-09-24,1.0
1,CITY:VE000001,"Barcelona, VE",1968-01-01,2024-09-24,1.0


In [17]:
pd.read_sql_query('''
SELECT distinct city, alpha_2
FROM results
where city = 'Barcelona'
limit 3
''', conn)

,city,alpha_2
0,Barcelona,ES


### merging in two steps
    1. only for cities that have only 1 uniqe country
    2. only cities with more then 1 uniqe countries

In [31]:
conn.execute('''
create table if not exists merged as
with part1 as (
SELECT *, SUBSTR(name, 1, INSTR(name, ',') - 1) as city
FROM locations
),
part2 as (
select city
from part1
group by 1
having count(*) = 1
),
part3 as (
select *
from results
where city in (select city from part2)
),
part4 as (
select t1.*, t2.id,
case when date between mindate and maxdate then 1 else 0 end as valid_date
from part3 t1
left join part1 t2
on t1.city = t2.city
),
part5 as (
select city
from part1
group by 1
having count(*) > 1
),
part6 as (
select *
from results
where city in (select city from part5)
),
part7 as (
select t1.*, t2.id,
case when date between mindate and maxdate then 1 else 0 end as valid_date
from part6 t1
left join part1 t2
on t1.city = t2.city and t1.alpha_2 = t2.alpha_2
),
part8 as (
select * from part4
union
select * from part7
)
select *
from part8
''')
conn.commit()

TypeError: 'NoneType' object is not iterable

### add to conn table with map between country name and country code

In [77]:
conn.execute('''
CREATE TABLE IF NOT EXISTS countries (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    alpha_2 TEXT NOT NULL
)
''')
conn.commit()

In [78]:
import pycountry
for country in pycountry.countries:
    cursor.execute('''
    INSERT INTO countries (name, alpha_2) VALUES (?, ?)
    ''', (country.name, country.alpha_2))

### updating results table by adding alpha_2 column from countries table

In [85]:
conn.execute('ALTER TABLE results ADD COLUMN alpha_2 TEXT')
conn.execute('''
UPDATE results
SET alpha_2 = (
    SELECT countries.alpha_2
    FROM countries
    WHERE results.country = countries.name
)
''')
conn.commit()

### updating results country name and alpha_2 code for better merge results

In [7]:
# first i will create a table with countries from results that do not merge with countries from pycountry
conn.execute('''
create table if not exists countries_not_merged as
select distinct country
from results
where alpha_2 is null
''')
conn.commit()

In [21]:
susu = pd.read_sql_query('''
select distinct country, alpha_2
from results
''', conn)
susu.to_csv('C:/Users/Mark Rozenberg/downloads/check3.csv')

In [22]:
susu = pd.read_sql_query('''
select distinct name, alpha_2
from countries
''', conn)
susu.to_csv('C:/Users/Mark Rozenberg/downloads/check4.csv')

### create mapping for countries

In [ ]:
pd.read_sql_query('''
select *
from results
where country like '%China%'
''', conn)

In [3]:
conn.execute('''
update results
set alpha_2 = 'UK'
where country in ('England', 'Scotland', 'Wales', 'Northern Ireland')
''')
conn.commit()

In [ ]:
conn.execute('''
update results
set country = 'China'
where country = 'China PR'
''')
conn.commit()

### Checks

In [6]:
fufu = pd.read_sql_query('''
select distinct country
from results
where alpha_2 is null
''', conn)
fufu

,country
0,Bohemia
1,Russia
2,China PR
3,Czechoslovakia
4,Yugoslavia
...,...
68,Northern Cyprus
69,Kosovo
70,East Timor
71,Tahiti


In [8]:
dudu = pd.read_sql_query('''
SELECT *
FROM countries
where name like '%boliv%'
''', conn)
dudu

,id,name,alpha_2
0,32,"Bolivia, Plurinational State of",BO
1,239,"Venezuela, Bolivarian Republic of",VE


In [ ]:
test = pd.read_sql_query('''
SELECT *
FROM results
where city like '%Manchester%'
''', conn)
test

In [ ]:
# join between locations and results (the city where the match played)
test = pd.read_sql_query('''
SELECT city
FROM results
group by 1
having count(distinct country) > 1
''', conn)
test

In [103]:
test = pd.read_sql_query('''
SELECT distinct country
FROM results
where city = 'Alexandria'
''', conn)
test

,country
0,United Arab Republic
1,Egypt


In [ ]:
gugu2 = pd.read_sql_query('''
select *
from Merged
where city_results = 'Tel Aviv'
order by date
''', conn)
gugu2

In [112]:
gugu3 = pd.read_sql_query('''
select *
from locations
where name like '%Tel Aviv%'
''', conn)
gugu3

,id,name,mindate,maxdate,datacoverage
0,CITY:IS000006,"Tel Aviv-Yafo, IS",1939-10-01,2024-09-24,1.0


In [68]:
test = pd.read_sql_query('''
with part1 as(
SELECT date, city_results, id
FROM Merged
where city_locations like '%Manchester%'
and id like '%UK%'
order by date desc
limit 2
),
part2 as(
SELECT date, city_results, id
FROM Merged
where city_locations like '%Liverpool%'
and id like '%UK%'
order by date desc
limit 2
)
select * from part1
union
select * from part2
''', conn)
test

,date,city_results,id
0,2001-03-24,Liverpool,CITY:UK000008
1,2002-04-17,Liverpool,CITY:UK000008
2,2018-03-23,Manchester,CITY:UK000010
3,2023-06-19,Manchester,CITY:UK000010


In [70]:
for index, row in test.iterrows():
    print(row['date'])
    print(row['id'])

2001-03-24
CITY:UK000008
2002-04-17
CITY:UK000008
2018-03-23
CITY:UK000010
2023-06-19
CITY:UK000010


### create table to hold climate data

In [47]:
conn.execute('''drop table if exists climate''')
conn.commit()

In [48]:
# Create the climate table if it doesn't exist
conn.execute('''
CREATE TABLE IF NOT EXISTS climate (
    date TEXT,
    datatype TEXT,
    station TEXT,
    attributes TEXT,
    value REAL
)
''')
conn.commit()

In [ ]:
import requests
import sqlite3
import json

# Define the endpoint
endpoint = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

# Set the headers with the API token
headers = {
    'token': api_token
}

In [45]:
# Define the parameters for the request
params = {
    'datasetid': 'GHCND',  # Global Historical Climatology Network - Daily
    'locationid': 'CITY:AE000001',
    'startdate': '2024-03-21',
    'enddate': '2024-03-21',
    'limit': 1000  # Number of results to return
}

# Make the request
response = requests.get(endpoint, headers=headers, params=params)
data = response.json()

# Connect to SQLite database (or create it if it doesn't exist)
# conn = sqlite3.connect('climate.db')
cursor = conn.cursor()

# Insert data into the climate table
for record in data['results']:
        cursor.execute('''
        INSERT OR REPLACE INTO climate (date, datatype, station, attributes, value)
        VALUES (?, ?, ?, ?, ?)
        ''', (record['date'], record['datatype'], record['station'], record['attributes'], record['value']))
        
# Commit the transaction and close the connection
conn.commit()

In [46]:
test = pd.read_sql_query('''
SELECT *
FROM climate
''', conn)
test

,date,datatype,station,attributes,value
0,2024-03-21T00:00:00,TAVG,GHCND:AEM00041217,"H,,S,",241.0
1,2024-03-21T00:00:00,TMAX,GHCND:AEM00041217,",,S,",275.0
2,2024-03-21T00:00:00,TAVG,GHCND:AEM00041217,"H,,S,",241.0
3,2024-03-21T00:00:00,TMAX,GHCND:AEM00041217,",,S,",275.0


In [32]:
params = {
    'datasetid': 'GHCND',  # Global Historical Climatology Network - Daily
    'locationid': 'CITY:AE000001',
    'startdate': '2024-03-21',
    'enddate': '2024-03-21',
    # 'datatypeid': 'TMAX'
}

# Make the request
response = requests.get(endpoint, headers=headers, params=params)
data = response.json()

for record in data['results']:
    print(record)

{'date': '2024-03-21T00:00:00', 'datatype': 'TAVG', 'station': 'GHCND:AEM00041217', 'attributes': 'H,,S,', 'value': 241}
{'date': '2024-03-21T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:AEM00041217', 'attributes': ',,S,', 'value': 275}
